In [1]:
from datetime import datetime, date, time
import numpy as np
import pandas as pd

## Data ingestion

In [2]:
file_path = '../data'

In [3]:
df = pd.read_excel(f'{file_path}/Shipment-Waybills-Oct24.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../data/Revised 23Sep-7Oct 24..xlsx'

In [ ]:
df.columns

Index(['Waybill Date', 'Waybill', 'Account', 'Shipper', 'Consignee',
       'Orig Hub', 'Orig Place', 'Dest Hub', 'Dest Place', 'Pieces', 'Service',
       'Chrg Mass', 'Booking Date', 'Actual Transit Days', 'POD Date',
       'POD Recipient', 'SLA Transit Days', 'Due Date', 'Last Event',
       'Last Event Date'],
      dtype='object')

## Data cleaning

In [ ]:
def safe_to_datetime(x):
    if pd.isna(x):
        return pd.NaT
    elif isinstance(x, (pd.Timestamp, np.datetime64, datetime)):
        return pd.to_datetime(x)
    elif isinstance(x, time):
        return pd.to_datetime(datetime.combine(date.today(), x))
    else:
        try:
            return pd.to_datetime(x)
        except:
            return pd.NaT

In [ ]:
date_columns = ['Due Date', 'Waybill Date', 'Last Event Date']
for col in date_columns:
    df[col] = df[col].apply(safe_to_datetime)

## Data manipulation

In [ ]:
# Build rules based on other columns

## Data output

In [ ]:
account_list = df['Account'].unique()
for account in account_list:
    df_account = df[df['Account'] == account]
    df_account = df_account.sort_values(by=['Last Event', 'Waybill Date'], ascending=[True, False])

    # Split the DataFrame
    df_not_pod = df_account[~df_account['Last Event'].isin(["POD Details Captured", "POD Image Scanned"])]
    df_pod = df_account[df_account['Last Event'].isin(["POD Details Captured", "POD Image Scanned"])]

    # Save to Excel file with multiple sheets
    with pd.ExcelWriter(f'{file_path}/output/{account}.xlsx') as writer:
        df_not_pod.to_excel(writer, sheet_name='Current deliveries', index=False)
        df_pod.to_excel(writer, sheet_name='Completed deliveries', index=False)